***Participantes (RM - NOME):***<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>

###**Criar um classificador de sentimento aplicando técnicas de PLN**
---

Utilizando o dataset de revisões de filmes em português [1], criar um classificador de sentimentos que consiga um score na métrica F1 Score superior a 70%.

Devem utilizar uma amostra de 20% e randon_state igual a 42 para testar as implementações e mensurar a métrica F1 Score (usar o parâmetro average = 'weighted') o restante dos dados devem ser utilizados para o treinamento (80%).

Fique a vontade para testar os métodos de pré-processamento, abordagens, algoritmos e bibliotecas, mas explique e justifique suas decisões.
O trabalho poderá ser feito em grupo de até 4 pessoas (mesmo grupo do Startup One).

Separe a implementação do seu modelo campeão junto com a parte de validação/teste de forma que o professor consiga executar todo o pipeline do modelo campeão.

Composição da nota:
- 50% - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, etc.)
- 50% - Baseado na performance obtida com o dataset de teste (conforme recomendação da amostra) no seu modelo campeão e na validação que o professor processar (Métrica F1 Score).

[1] - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv

Bom desenvolvimento!

In [1]:
import nltk
import spacy

import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import f1_score


C:\Users\Davidson\AppData\Local\Temp/ipykernel_13060/2353454606.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
# CARREGANDO O DATA FRAME
df = pd.read_csv('data/reviews-pt-br.csv', encoding='utf-8')

df = df.sample(2000)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 32887 to 5298
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   codigo      2000 non-null   int64 
 1   texto       2000 non-null   object
 2   sentimento  2000 non-null   object
dtypes: int64(1), object(2)
memory usage: 62.5+ KB


In [4]:
# Carrega StopWords
nlp = spacy.load('pt_core_news_sm')
nltk.download('stopwords')

# stopwords NLTK
stops_nltk = nltk.corpus.stopwords.words('portuguese')

# stopwords SpaCy
stops_spacy = nlp.Defaults.stop_words

# stopwords do SpaCy e NLTK combinadas
stops = list(set(stops_spacy).union(set(stops_nltk)))
len(stops)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Davidson\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


499

In [8]:
# função de lematização dos verbos do documento
def fn_lematiza_verb_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        if word.pos_ =='VERB':
            sent.append(word.lemma_)
        else:
            sent.append(word.text)
    return " ".join(sent)

# função para limpar documento
def fn_limpa_texto(texto):
    result = []
    texto = re.findall(r"[a-zA-z]+", texto)
    for w in texto:
        w = w.lower()
        if w in stops:
            continue
        result.append(w)
        
    texto = ' '.join(result)
    return texto


In [10]:
df.dropna(inplace=True)
# limpa texto
df['texto_trat'] = df.texto.progress_apply(fn_limpa_texto)

# aplica a lematização dos verbos no dataframe
df['texto_lemma_verb'] = df.texto_trat.progress_apply(fn_lematiza_verb_texto)

df['sentimento'] = df.sentimento.progress_apply(lambda x: 1 if x == 'pos' else 0)

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [11]:
df.head(20)

,codigo,texto,sentimento,texto_trat,texto_lemma_verb
32887,32888,outro multid babes bullets hist ria ousada out...,1,outro multid babes bullets hist ria ousada out...,outro multid babes bullets hist rir ousar outr...
17716,17717,pena ningu m conhecer pequena obra prima suspe...,1,pena ningu m conhecer pequena obra prima suspe...,pena ningu m conhecer pequena obra prima suspe...
8456,8457,haver crian crescido terem visto nica nica bug...,0,haver crian crescido terem visto nica nica bug...,haver crian crescer terem vestir nica nica bug...
3051,3052,ok concordaram melhor temporada matar boone m ...,1,ok concordaram melhor temporada matar boone m ...,ok concordar melhor temporada matar boone m de...
31202,31203,blockbuster baseketball daqueles filmes costum...,0,blockbuster baseketball daqueles filmes costum...,blockbuster baseketball daqueles filmes costum...
14118,14119,classificado vida real londres ent produtores ...,0,classificado vida real londres ent produtores ...,classificado vida real londres ent produtores ...
32646,32647,first blood tenta quest tratamento veteranos v...,0,first blood tenta quest tratamento veteranos v...,first blood tentar quest tratamento veteranos ...
5227,5228,filme controv rsias escritas projetos verhoeve...,1,filme controv rsias escritas projetos verhoeve...,filme controv rsias escrito projetos verhoeven...
19987,19988,acho maiores filmes burts junto deliverance sm...,1,acho maiores filmes burts junto deliverance sm...,achar maiores filmes burts junto deliverance s...
14913,14914,pesadelo insomniacs hist ria homem mergulhar i...,1,pesadelo insomniacs hist ria homem mergulhar i...,pesadelo insomniacs hist rir homem mergulhar i...


In [12]:
# pré-processamento vetorização e BOW
# Frequência de termos - term frequency–inverse document frequency (TF-IDF)
vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=True) #0.7729036810537457 - TF-IDF, texto lematixado, uni e bi brama, Naive

In [13]:
# texto tratado e verbos lematizado
vetor.fit(df.texto_lemma_verb)
text_vect = vetor.transform(df.texto_lemma_verb)

In [14]:
# separa as amostras de Treino (80%) e Texte (20%)
X_train,X_test,y_train,y_test = train_test_split(text_vect, df['sentimento'], test_size = 0.2, random_state = 42)
print(text_vect.shape)

(2000, 187988)


In [15]:
# define e treina o modelo de Logistic Regression
modelo_lr = LogisticRegression(random_state=42)

modelo_lr.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo_lr.predict(X_test)
logistic_regression_accuracy = accuracy_score(y_prediction, y_test)

In [16]:
# define e treina o modelo de Logistic Regression com penalty
modelo_lrp = LogisticRegression(penalty = 'l2', C = 100, random_state=42)

modelo_lrp.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo_lrp.predict(X_test)
logistic_regression_penalty_accuracy = f1_score(y_prediction, y_test, average='weighted')

In [17]:
# define e treina o modelo de classificação Decision Tree Classifier
modelo_dtc = DecisionTreeClassifier(random_state=42)

modelo_dtc.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo_dtc.predict(X_test)
decision_tree_accuracy = f1_score(y_prediction, y_test, average='weighted')

In [18]:
# define e treina o modelo de Multinomial
modelo_m = MultinomialNB()

modelo_m.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo_m.predict(X_test)
multinomial_accuracy = f1_score(y_prediction, y_test, average='weighted')

In [19]:
print(f'{logistic_regression_accuracy=}')
print(f'{logistic_regression_penalty_accuracy=}')
print(f'{decision_tree_accuracy=}')
print(f'{multinomial_accuracy=}')

logistic_regression_accuracy=0.805
logistic_regression_penalty_accuracy=0.8232866211522338
decision_tree_accuracy=0.6474007197621655
multinomial_accuracy=0.7919006913890635


####**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.